<a href="https://colab.research.google.com/github/gyuchangShim/yolo_nas_practice/blob/main/custom_dataset_yolo_nas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install super-gradients
!pip install roboflow

In [ ]:
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train,
    coco_detection_yolo_format_val
)
from super_gradients.training import models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import (
    DetectionMetrics_050,
    DetectionMetrics_050_095
)
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from tqdm.auto import tqdm

import os
import requests
import zipfile
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
import random

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="aIU6ug3LhFpJCcpMteMN")
project = rf.workspace("mjuaiimages").project("mju_ai_week02")
version = project.version(1)
dataset = version.download("yolov5")

In [ ]:
# dataset_setup
ROOT_DIR = '/content/mju_ai_week02-1'
train_imgs_dir = 'images/train'
train_labels_dir = 'labels/train'
val_imgs_dir = 'images/val'
val_labels_dir = 'labels/val'
test_imgs_dir = 'images/test'
test_labels_dir = 'labels/test'
classes = ['cat', 'dog']

dataset_params = {
    'data_dir': train_imgs_dir,
    'train_images_dir': train_imgs_dir,
    'train_labels_dir': train_labels_dir,
    'val_images_dir': val_imgs_dir,
    'val_labels_dir': val_labels_dir,
    'test_images_dir': test_imgs_dir,
    'test_labels_dir': test_labels_dir,
    'classes': classes
}

In [ ]:
# Global parameters.
EPOCHS = 5
BATCH_SIZE = 16
WORKERS = 8